In [2]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    if "google.colab" in sys.modules:
        !pip install -q pyomo
    else:
        !conda install -c conda-forge pyomo
    assert(shutil.which("pyomo"))
    
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass
    assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

import pyomo.environ as pe

Mettre en place le modèle **compact** suivant avec *pyomo*:

\begin{align}
\text{minimiser} &\sum_{i \in I} \sum_{j \in J} c_{ij}w_{ij} + \omega c_{ij}x_{ij} + 2\omega c_{ij}y_{ij} \\
\text{Sujet à} &\sum_{j \in J} w_{ij} + x_{ij} + y_{ij} \leq X_i &&\forall i \in I \text{ (inventory)}\\
&\sum_{i \in I} w_{ij} = D_j (1 - \lambda_j) &&\forall j \in J \text{ (single_demand)}\\
&\sum_{i \in I} x_{ij} + y_{ij} = D_j \lambda_j &&\forall j \in J \text{ (multi_demand)}\\
&x_{ij} \leq \rho_i D_j \lambda_j &&\forall i \in I, j \in J \text{ (shipment)}\\
&w_{ij}, x_{ij}, y_{ij} \geq 0 &&\forall i \in I, j \in J \\
\end{align}

avec les paramètres suivants:

In [3]:
# ensembles
I = ['Utah', 'Nevada']
J = ['Kansas']

# paramètres
# c_ij
c = {('Utah', 'Kansas'): 9,
     ('Nevada', 'Kansas'): 12}
# omega
Omega = 0.33
# X_i
X = {'Utah': 5,  
     'Nevada': 20}
# rho_i
Rho = {'Utah': 0.5,
       'Nevada': 0.2}
# D_j
D = {'Kansas': 20}
# lambda_j
Lambda = {'Kansas': 0.75}

# Modèle détaillé

Pour vous aider dans cette tâche, vous avez accès au modèle détaillé suivant:

In [4]:
model = pe.ConcreteModel()

# variables
model.w11 = pe.Var(within=pe.NonNegativeReals)  # (Utah, Kansas)
model.w21 = pe.Var(within=pe.NonNegativeReals)  # (Nevada, Kansas)
model.x11 = pe.Var(within=pe.NonNegativeReals)  # (Utah, Kansas)
model.x21 = pe.Var(within=pe.NonNegativeReals)  # (Nevada, Kansas)
model.y11 = pe.Var(within=pe.NonNegativeReals)  # (Utah, Kansas)
model.y21 = pe.Var(within=pe.NonNegativeReals)  # (Nevada, Kansas)

# objectif
model.obj = pe.Objective(
    sense=pe.minimize,
    expr=9*model.w11 + 12*model.w21
         + 0.33*9*model.x11 + 0.33*12*model.x21
         + 2*0.33*9*model.y11 + 2*0.33*12*model.y21)

# contraintes 1
model.inventory1 = pe.Constraint(
    expr=model.w11 + model.x11 + model.y11 <= 5) 
model.inventory2 = pe.Constraint(
    expr=model.w21 + model.x21 + model.y21 <= 20) 

# contraintes 2
model.single_demand1 = pe.Constraint(
    expr=model.w11 + model.w21 == 20 * (1 - 0.75))

# contraintes 3
model.multi_demand1 = pe.Constraint(
    expr=model.x11 + model.x21 + model.y11 + model.y21 == 20 * 0.75)

# contraintes 4
model.shipment1 = pe.Constraint(
    expr=model.x11 <= 20 * 0.75 * 0.5)
model.shipment2 = pe.Constraint(
    expr=model.x21 <= 20 * 0.75 * 0.2)

pe.SolverFactory('glpk').solve(model).write()
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 142.17
  Upper bound: 142.17
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 6
  Number of nonzeros: 14
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.035601139068603516
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

# Modèle compact

In [5]:
model = pe.ConcreteModel()

# variables
model.w = pe.Var(I, J, within=pe.NonNegativeReals)
model.x = pe.Var(I, J, within=pe.NonNegativeReals)
model.y = pe.Var(I, J, within=pe.NonNegativeReals)

# objectif
model.obj = pe.Objective(
    sense=pe.minimize,
    expr=sum(c[i,j]*model.w[i,j]
             + Omega*c[i,j]*model.x[i,j]
             + 2*Omega*c[i,j]*model.y[i,j]
             for i in I for j in J))

# contraintes 1
model.inventory = pe.ConstraintList()
for i in I:
    model.inventory.add(
        expr=sum(model.w[i,j] + model.x[i,j] + model.y[i,j] 
                 for j in J) <= X[i])

# contraintes 2
model.single_demand = pe.ConstraintList()
for j in J:
    model.single_demand.add(
        expr=sum(model.w[i,j] for i in I) == D[j]*(1-Lambda[j]))

# contraintes 3
model.multi_demand = pe.ConstraintList()
for j in J:
    model.multi_demand.add(
        expr=sum(model.x[i,j] + model.y[i,j] 
                 for i in I) == D[j]*Lambda[j])

# contraintes 4
model.shipment = pe.ConstraintList()
for i in I:
    for j in J:
        model.shipment.add(model.x[i,j] <= D[j]*Lambda[j]*Rho[i])

pe.SolverFactory('glpk').solve(model).write()
model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 142.17
  Upper bound: 142.17
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 6
  Number of nonzeros: 14
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.04816389083862305
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------